In [1]:
from tomoSegmentPipeline.utils.common import read_array, write_array
from tomoSegmentPipeline.utils import setup
from cryoS2Sdrop.dataloader import singleCET_dataset
from cryoS2Sdrop.model import Denoising_UNet

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torchsummary import summary

import os
from torch.utils.data import Dataset, DataLoader

PARENT_PATH = setup.PARENT_PATH
ISONET_PATH = os.path.join(PARENT_PATH, 'data/isoNet/')

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
cet_path = os.path.join(PARENT_PATH, 'data/raw_cryo-ET/tomo02.mrc') 
p = 0.3
n_samples = 5
my_dataset = singleCET_dataset(cet_path, 128, p=p, n_samples=n_samples)

In [3]:
dloader = DataLoader(my_dataset, batch_size=1, shuffle=True, pin_memory=True)

In [4]:
for batch in dloader:
    subtomo, target, mask = batch
    print(subtomo.shape)
    break

torch.Size([1, 5, 128, 128, 128])


In [12]:
n_features = 48
model = Denoising_UNet(None, 0, n_samples, n_features, p)

In [15]:
model(subtomo).shape

torch.Size([1, 5, 128, 128, 128])

In [14]:
summary(model, (5, 128, 128, 128), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
     PartialConv3d-1    [-1, 48, 128, 128, 128]           6,528
     PartialConv3d-2    [-1, 48, 128, 128, 128]          62,256
         LeakyReLU-3    [-1, 48, 128, 128, 128]               0
         MaxPool3d-4       [-1, 48, 64, 64, 64]               0
     PartialConv3d-5       [-1, 48, 64, 64, 64]          62,256
         LeakyReLU-6       [-1, 48, 64, 64, 64]               0
         MaxPool3d-7       [-1, 48, 32, 32, 32]               0
     PartialConv3d-8       [-1, 48, 32, 32, 32]          62,256
         LeakyReLU-9       [-1, 48, 32, 32, 32]               0
        MaxPool3d-10       [-1, 48, 16, 16, 16]               0
    PartialConv3d-11       [-1, 48, 16, 16, 16]          62,256
        LeakyReLU-12       [-1, 48, 16, 16, 16]               0
        MaxPool3d-13          [-1, 48, 8, 8, 8]               0
    PartialConv3d-14          [-1, 48, 